In [1]:
import requests
import re
from bs4 import BeautifulSoup
import datetime as dt

In [2]:
url = "http://portal.iprachicago.org/wp-content/themes/ipra/DynamicSearch.php"
headers = {
    'user-agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"
}

r = requests.get(url, headers=headers)
print("Status Code: ", r.status_code)

Status Code:  200


In [3]:
r.text[0:100]

'{"caseSearch":{"items":"\\u003Cdiv class=\\u0022dynamic-search-loading-container\\u0022\\u003E\\n\\t\\t\\u00'

In [4]:
r.headers

{'Vary': 'Accept-Encoding,Cookie', 'Server': 'nginx', 'Date': 'Thu, 21 Jul 2016 20:39:16 GMT', 'X-Cache-Group': 'normal', 'Content-Type': 'application/json', 'Accept-Ranges': 'bytes', 'X-Type': 'default', 'X-Cacheable': 'SHORT', 'Connection': 'keep-alive', 'X-Cache': 'MISS', 'Keep-Alive': 'timeout=20', 'X-Pass-Why': '', 'Cache-Control': 'max-age=600, must-revalidate', 'Content-Length': '72944'}

In [5]:
content = r.json()['caseSearch']['items']

In [6]:
content

'<div class="dynamic-search-loading-container">\n\t\t<div class="loading-animation">\n\t\t\t<img src="http://portal.iprachicago.org/wp-content/themes/ipra/img/loadbar.gif" alt="Loading" />\n\t\t</div>\n\t</div>\n\n\t<table class="table table-striped" id="ipra-case-search-data-table">\n\t    <thead>\n\t        <tr>\n\t            <th>Log#</th>\n\t            <th>Incident Types</th>\n\t            <th>IPRA Notification Date</th>\n\t            <th>Incident Date & Time</th>\n\t        </tr>\n\t    </thead>\n\t    <tbody>\n\t\t\t        <tr>\n\t            <th>\n\t            \t<a href="http://portal.iprachicago.org/1062377-2/" rel="bookmark">\n\t\t\t\t\t\t1062377\t\t\t\t\t</a>\n\t            </th>\n\t            <td>\n\t\t\t\t\tIncident in Police Custody\t           \t</td>\n\t            <td>\n\t            \t\t            \t\t05-21-2013\t\t\t\t\t\t            </td>\n\t            <td>\n\t            \t\t            \t\t05-13-2013 11:00 pm\t\t\t\t\t\t\t\t\t</td>\n\t        </tr>\n\t\t\t 

In [7]:
content = re.sub(r'\n','',content)
content = re.sub(r'\t','',content)
content

'<div class="dynamic-search-loading-container"><div class="loading-animation"><img src="http://portal.iprachicago.org/wp-content/themes/ipra/img/loadbar.gif" alt="Loading" /></div></div><table class="table table-striped" id="ipra-case-search-data-table">    <thead>        <tr>            <th>Log#</th>            <th>Incident Types</th>            <th>IPRA Notification Date</th>            <th>Incident Date & Time</th>        </tr>    </thead>    <tbody>        <tr>            <th>            <a href="http://portal.iprachicago.org/1062377-2/" rel="bookmark">1062377</a>            </th>            <td>Incident in Police Custody           </td>            <td>                        05-21-2013            </td>            <td>                        05-13-2013 11:00 pm</td>        </tr>        <tr>            <th>            <a href="http://portal.iprachicago.org/1066371-2/" rel="bookmark">1066371</a>            </th>            <td>Incident in Police Custody           </td>            <td

In [8]:
soup = BeautifulSoup(content, 'html.parser')

In [9]:
soup

<div class="dynamic-search-loading-container"><div class="loading-animation"><img alt="Loading" src="http://portal.iprachicago.org/wp-content/themes/ipra/img/loadbar.gif"/></div></div><table class="table table-striped" id="ipra-case-search-data-table"> <thead> <tr> <th>Log#</th> <th>Incident Types</th> <th>IPRA Notification Date</th> <th>Incident Date &amp; Time</th> </tr> </thead> <tbody> <tr> <th> <a href="http://portal.iprachicago.org/1062377-2/" rel="bookmark">1062377</a> </th> <td>Incident in Police Custody           </td> <td>                        05-21-2013            </td> <td>                        05-13-2013 11:00 pm</td> </tr> <tr> <th> <a href="http://portal.iprachicago.org/1066371-2/" rel="bookmark">1066371</a> </th> <td>Incident in Police Custody           </td> <td>                        12-02-2013            </td> <td>                        07-25-2013 10:04 pm</td> </tr> <tr> <th> <a href="http://portal.iprachicago.org/1080615-2/" rel="bookmark">1080615</a> </th> <

In [10]:
table = soup.find('table', attrs={'id':'ipra-case-search-data-table'})

rows = table.find_all('tr')

In [11]:
rows[0]

<tr> <th>Log#</th> <th>Incident Types</th> <th>IPRA Notification Date</th> <th>Incident Date &amp; Time</th> </tr>

In [12]:
rows[1]

<tr> <th> <a href="http://portal.iprachicago.org/1062377-2/" rel="bookmark">1062377</a> </th> <td>Incident in Police Custody           </td> <td>                        05-21-2013            </td> <td>                        05-13-2013 11:00 pm</td> </tr>

In [13]:
rows[1].find('th').find('a').string

'1062377'

In [14]:
rows[1].find('th').find('a').get('href')

'http://portal.iprachicago.org/1062377-2/'

In [15]:
tdlist = rows[1].find_all('td')
tdlist

[<td>Incident in Police Custody           </td>,
 <td>                        05-21-2013            </td>,
 <td>                        05-13-2013 11:00 pm</td>]

In [16]:
tdlist[2].string.strip()

'05-13-2013 11:00 pm'

In [17]:
len(rows)

119

In [18]:
data = {'records' : {},
       'scraped_date_time' : dt.datetime.now().isoformat()}
for row in rows[1:]:
    
    th = row.find('th')
    logno = th.find('a').string.strip()
    
    record = {}
    record['record_url'] = th.find('a').get('href').strip()
    
    tdlist = row.find_all('td')
    
    record['incident_type'] = tdlist[0].string.strip()
    record['ipra_notification_date'] = tdlist[1].string.strip()
    record['incident_date_time'] = tdlist[2].string.strip()
    
    data['records'][logno] = record

In [19]:
data['records']

{'1042469': {'incident_date_time': '01-04-2011 9:35 pm',
  'incident_type': 'Firearm Discharge',
  'ipra_notification_date': '01-04-2011',
  'record_url': 'http://portal.iprachicago.org/1042469-2/'},
 '1042532': {'incident_date_time': '01-07-2011 1:33 am',
  'incident_type': 'Firearm Discharge',
  'ipra_notification_date': '01-07-2011',
  'record_url': 'http://portal.iprachicago.org/1042532-2/'},
 '1044336': {'incident_date_time': '03-29-2011 7:48 am',
  'incident_type': 'Firearm Discharge',
  'ipra_notification_date': '03-29-2011',
  'record_url': 'http://portal.iprachicago.org/1044336-2/'},
 '1045673': {'incident_date_time': '05-25-2011 10:30 pm',
  'incident_type': 'Firearm Discharge',
  'ipra_notification_date': '05-25-2011',
  'record_url': 'http://portal.iprachicago.org/1045673-2/'},
 '1045950': {'incident_date_time': '06-07-2011 1:48 am',
  'incident_type': 'Firearm Discharge',
  'ipra_notification_date': '06-07-2011',
  'record_url': 'http://portal.iprachicago.org/1045950-2/'},

In [20]:
data = []
table = soup.find('table', attrs={'class':'lineItemsTable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

AttributeError: 'NoneType' object has no attribute 'find'

# Scrape pages

In [21]:
url = 'http://portal.iprachicago.org/1042469-2/'
r = requests.get(url, headers=headers)

In [22]:
soup = BeautifulSoup(r.content,'html.parser')

In [23]:
content = soup.find(id='main')
content

<main class="site-main" id="main" role="main"><article class="post-173 post type-post status-publish format-standard hentry category-firearm-discharge" id="post-173"><header class="entry-header"><h1 class="entry-title">1042469</h1><div class="entry-meta"> <span class="posted-on">Posted on <a href="http://portal.iprachicago.org/1042469-2/" rel="bookmark" title="1:47 am"><time class="entry-date published" datetime="2016-06-03T01:47:12+00:00">June 3, 2016</time></a> </span> <span class="byline"> by <span class="author vcard"><a class="url fn n" href="http://portal.iprachicago.org/author/s-kenney/" title="View all posts by IPRA">IPRA</a></span></span></div></header><div class="entry-content"><div class="table-responsive hidden-lg hidden-md stack-table"><table class="table table-bordered table-striped"><tbody><tr><th scope="row">Log Number:</th><td>1042469</td></tr><tr><th scope="row">Case Type(s):</th><td> Firearm Discharge</td></tr><tr><th scope="row">IPRA Notification Date:</th><td> 01-0

In [24]:
content.find('time',attrs={'class' : 'entry-date published'}).get("datetime")

'2016-06-03T01:47:12+00:00'

In [25]:
entry = content.find('div',attrs={'class' : 'entry-content'})
entry

<div class="entry-content"><div class="table-responsive hidden-lg hidden-md stack-table"><table class="table table-bordered table-striped"><tbody><tr><th scope="row">Log Number:</th><td>1042469</td></tr><tr><th scope="row">Case Type(s):</th><td> Firearm Discharge</td></tr><tr><th scope="row">IPRA Notification Date:</th><td> 01-04-2011</td></tr><tr><th scope="row">Incident Date &amp; Time:</th><td> 01-04-2011 9:35 pm</td></tr></tbody></table></div><div class="table-responsive hidden-sm hidden-xs"><table class="table table-striped"><thead><tr><th scope="col">Log#:</th><th scope="col">Incident Type(s):</th><th scope="col">IPRA Notification Date:</th><th scope="col">Incident Date &amp; Time:</th></tr></thead><tbody><tr><th scope="row"> 1042469</th><td> Firearm Discharge</td><td> 01-04-2011</td><td> 01-04-2011 9:35 pm</td></tr></tbody></table></div><p>Subject:  Jermaine Henry</p><p>Pursuant to the Illinois Juvenile Court Act, 705 ILCS 405/1-1, <em>et seq.</em>, the City is prohibited from r

In [26]:
rows = entry.find_all('div',attrs={'class' : 'row'})

In [27]:
cols = entry.find_all('div',attrs={'class' : 'col-sm-4'})

In [28]:
cols[1]

IndexError: list index out of range

In [29]:
cols[30].find('span',attrs={'class':'fa'}).get('class')

IndexError: list index out of range

In [ ]:
'fa-file-pdf-o' in cols[30].find('span',attrs={'class':'fa'}).get('class')

In [ ]:
cols[10]

In [ ]:
cols[10].find('div',attrs={'class':'large-icon'}).text.strip()

In [ ]:
js = cols[1].find('script').text
js = re.sub(r'\n','',js)
js = re.sub(r'\t','',js)
js

In [ ]:
re.search(r"var\s+src\s*=\s*'([^']+)'", js).groups()[0]

In [ ]:
re.search(r"'#modal(\d+)'",js).groups()[0]

In [31]:
note = entry.find('p',string=re.compile(r"Note:"))
note

In [32]:
subject = entry.find('p',string=re.compile(r"Subject:"))
subject 

<p>Subject:  Jermaine Henry</p>

In [ ]:
re_subjectcapture = re.compile(r"Subject:\s*(.*)")
re.search(re_subjectcapture,subject.string).groups()[0].strip()

In [30]:
entry.ul.find_all('li')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
re_juvnotefind = re.compile(r"S")
entry.find_all('p',contents=re_juvnotefind,recursive=True)

In [ ]:
re_juvnotefind = re.compile(r"P")
for p in entry.find_all('p'):
    if re_juvnotefind.search(p.text):
        print(p.text.strip())

In [ ]:
entry.find_all('p')[1].text

In [33]:
# Compiled regex
re_notefind = re.compile(r"Note:")
re_notecapture = re.compile(r"Note:\s*(.*)")
re_juvnotefind = re.compile(r"Pursuant")
re_subjectcapture = re.compile(r"Subject[^<:]*:([^<:]+)")

re_fileid = re.compile(r"'#modal(\d+)'")
re_mediasrc = re.compile(r"var\s+src\s*=\s*'([^']+)'")



for logno, record in data['records'].items():
    
    r = requests.get(record['record_url'], headers=headers)
    soup = BeautifulSoup(r.content,'html.parser')
    
    # Convenient sections
    content = soup.find(id='main')
    entry = content.find('div',attrs={'class' : 'entry-content'})
    
    # Get date and time posted on IPRA portal
    record['post_date_time'] = content.find('time',attrs={'class' : 'entry-date published'}).get("datetime")
    
    # Get notes
    record['notes'] = []
    # If "Note:" exists
    note = entry.find('p',string=re_notefind)
    if note:
        record['notes'].append(re_notecapture.search(note.string).groups()[0].strip())
    # If Illinois Juvenile Court Act note exists
    for p in entry.find_all('p'):
        if re_juvnotefind.search(p.text):
            record['notes'].append(p.text.strip())
    
    # Subjects
    record['subjects'] = []
    # Check if there is an unordered list of multiple subjects
    subjectlist = entry.find('ul')
    if subjectlist:
        for subject in subjectlist.find_all('li'):
            record['subjects'].append(subject.string.strip())
    # Else check for single line subject
    elif entry.find('p',string=re_subjectcapture):
        subjectline = entry.find('p',string=re_subjectcapture).string
        subjectlist.append(re_subjectcapture.search(subjectline).groups()[0].strip())
    '''
    # Get single Subject if exists
    subject = entry.find('p',string=re_subjectfind)
    if subject:
        record['subjects'] = [re_subjectcapture.search(subject.string).groups()[0].strip()]
    # Get multiple Subjects if exists
    if entry.find('p',string=re_subjectsmultfind):
            record['subjects'] = [li.string.strip() for li in entry.ul.find_all('li')]
        
    # Get multiple Subjects if exists
    if entry.find('p',string=re_subjectsmultfind):
        record['subjects'] = [li.string.strip() for li in entry.ul.find_all('li')]
    '''
    
    
    # Scrape media
    media = entry.find_all('div',attrs={'class' : 'col-sm-4'})
    
    record['documents'] = []
    record['audio'] = []
    record['video'] = []
    
    for elem in media:
        iconclasses = elem.find('span',attrs={'class':'fa'}).get('class')
        
        # If document
        if 'fa-file-pdf-o' in iconclasses:
            record['documents'].append(
                {'description' : elem.a.text.strip(), 
                'url' : elem.a.get('href').strip()}
                )
        
        # If video
        elif 'fa-file-video-o' in iconclasses:
            descript = elem.find('div',attrs={'class':'large-icon'}).text.strip()
            
            js = elem.find('script').text
            js = re.sub(r'\n','',js)
            js = re.sub(r'\t','',js)
            
            file_id = re_fileid.search(js).groups()[0]
            
            url = re_mediasrc.search(js).groups()[0]
            
            record['video'].append({
                    'description' : descript, 
                    'file_id' : file_id,
                    'url' : url})
            
        # If audio
        elif 'fa-file-sound-o' in iconclasses:
            descript = elem.find('div',attrs={'class':'large-icon'}).text.strip()
            
            js = elem.find('script').text
            js = re.sub(r'\n','',js)
            js = re.sub(r'\t','',js)
            
            file_id = re_fileid.search(js).groups()[0]
            
            url = re_mediasrc.search(js).groups()[0]
            
            record['audio'].append({
                    'description' : descript, 
                    'file_id' : file_id,
                    'url' : url})

TypeError: list indices must be integers or slices, not str

In [ ]:
data['records']['1052578']